In [1]:
import numpy as np
#%pip install simpy seaborn matplotlib
import simpy
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Parameters
LAMBDA = 1.0  # Average arrival rate (customers per time unit)
MU = 0.1
RHO = LAMBDA / MU  # Utilization factor
SIM_TIME = 10000  # Simulation time in number of time units

times_in_system = []  # List to record time spent in system for each customer
num_customers_in_system = []  # Number of customers in the system at regular intervals



In [3]:
# calculation of expected lenght and waiting time in the system 
LS = RHO/(1-RHO) # Length of the system
WS = LS/LAMBDA # Waiting time in the system

Expected number of customers in the system: -1.1111111111111112
Expected time a customer spends in the system: -1.1111111111111112


In [4]:
# simulation processes
def customer(env, name, server):
    """Customer process. Customers arrive and get served."""
    arrival_time = env.now

    # Request a server
    with server.request() as req:
        yield req

        # Server is available, start service
        service_time = random.expovariate(MU)
        yield env.timeout(service_time)
        
        # Record the time spent in the system
        times_in_system.append(env.now - arrival_time)

def source(env, server):
    """Generate new customers."""
    i = 0
    while True:
        yield env.timeout(random.expovariate(LAMBDA)) # Generate new customers at random times
        i += 1
        env.process(customer(env, f"Customer-{i}", server))

def monitor(env, server, interval=1):
    """Monitor the number of customers in the system at regular intervals."""
    while True:
        num_customers_in_system.append(len(server.queue) + server.count)  # server.count is the number of customers currently being served
        yield env.timeout(interval)

In [10]:
# Setup and start the simulation
reached = False
MU = 0.1
quantile_90 = np.float16(0)
while not reached:
    random.seed(42)
    env = simpy.Environment()
    
    # Start processes and run
    server = simpy.Resource(env, capacity=1)
    env.process(source(env, server))
    env.process(monitor(env, server))
    env.run(until=SIM_TIME)
    quantile_90 = np.quantile(times_in_system, 0.9)
    if quantile_90 < np.float16(1):
        reached = True
    else:
        print(f'Got 90th percentile of {quantile_90}, with service rate {MU}')
        MU += 0.05
        times_in_system = []  # List to record time spent in system for each customer
        num_customers_in_system = []  # Number of customers in the system at regular intervals

print("Adjusted service rate: {}".format(MU))
print("90th percentile of time spent in the system: {}".format(quantile_90))

Got 90th percentile of 1.813789061661737, with service rate 0.1
Got 90th percentile of 7771.557936687285, with service rate 0.15000000000000002
Got 90th percentile of 7240.768654421324, with service rate 0.2
Got 90th percentile of 6706.64328911752, with service rate 0.25
Got 90th percentile of 6250.583888477268, with service rate 0.3
Got 90th percentile of 5850.177118607894, with service rate 0.35
Got 90th percentile of 5541.525692958584, with service rate 0.39999999999999997
Got 90th percentile of 5121.260724824652, with service rate 0.44999999999999996
Got 90th percentile of 4384.935973191263, with service rate 0.49999999999999994
Got 90th percentile of 4088.9110069483777, with service rate 0.5499999999999999
Got 90th percentile of 3588.704441018304, with service rate 0.6
Got 90th percentile of 3172.2128516780017, with service rate 0.65
Got 90th percentile of 2888.2462165037355, with service rate 0.7000000000000001
Got 90th percentile of 2298.4226879541093, with service rate 0.750000

In [ ]:
# Set the Seaborn style
sns.set_style("whitegrid")

# Compute the average time in system
avg_time = sum(times_in_system) / len(times_in_system)

# Plotting the times in system
plt.figure(figsize=(12, 7))
sns.lineplot(x=range(len(times_in_system)), y=times_in_system, marker='o', linestyle='-', label='Time Spent in System')
plt.axhline(avg_time, color='red', linestyle='--', label=f'Average: {avg_time:.2f}')
plt.title("Time Spent in System by Each Customer")
plt.xlabel("Customer")
plt.ylabel("Time in System")
plt.legend()
plt.show()

In [ ]:
# create a histogram of the time spent in the system use 0.2 as the bin size
plt.figure(figsize=(12, 7))
sns.histplot(x=times_in_system, binwidth=0.2, stat="probability", color='red')
plt.title("Time Spent in System by Each Customer")
plt.xlabel("Time in System")
plt.ylabel("Probability")
plt.show()


In [ ]:
# compare expected and simulated average time in system
print("simulated average time in system: {}".format(avg_time))
print("calculated average time in system: {}".format(WS))

In [ ]:
# Plotting the number of customers in the system
plt.figure(figsize=(12, 7))
sns.lineplot(x=range(len(num_customers_in_system)), y=num_customers_in_system, marker='o', linestyle='-', label='Number of Customers')
avg_customers = sum(num_customers_in_system) / len(num_customers_in_system)
plt.axhline(avg_customers, color='red', linestyle='--', label=f'Average Customers: {avg_customers:.2f}')
plt.title("Number of Customers in the System Over Time")
plt.xlabel("Time")
plt.ylabel("Number of Customers")
plt.legend()
plt.show()

In [ ]:
# compare simulated and expected average number of customers in the system
print("simulated average customers in system: {}".format(avg_customers))
print("calculated average customers in system: {}".format(LS))

In [ ]:
# histogram of numof customers in the system, x-axis ticks should be each integer
plt.figure(figsize=(12, 7))
sns.histplot(x=num_customers_in_system, stat="probability", color='red', discrete=True)
plt.title("Number of Customers in the System")
plt.xlabel("Number of Customers")
plt.ylabel("Probability")
plt.xticks(range(max(num_customers_in_system)+1))
plt.show()
